In [1]:
import bs4
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import json
import re
import lxml.html
import time
import random
from random import randint
import logging
import collections
from time import gmtime, strftime

import re
from tabulate import tabulate
import os

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import typing
from typing import List, Dict

In [59]:
driver = webdriver.Firefox()

driver.get("https://immoelissa.be/annonces-immobilieres/4295347-rez-de-chaussee-commercial-3/")


In [60]:
price = driver.find_element_by_class_name("price")
print(price.text)

105 000


In [61]:
detailsKeys = []
detailsValues = []
details = {}

for desc_list in driver.find_elements_by_tag_name("dt"):
    detail = desc_list.text.split("\n")
    detailsKeys.extend(detail)
    
for desc_list in driver.find_elements_by_tag_name("dd"):
    detail = desc_list.text.split("\n")
    detailsValues.extend(detail)
    
for idx, x in enumerate(detailsKeys):
    details[x] = detailsValues[idx]
    
print(details)
        
    

{'Prix': '105 000 €', 'Référence': '4045413', 'Ville': 'Liege', 'Type du bien': 'Commerce', 'Disponibilité': 'Vendu', 'État du bien': 'Bon état', 'Revenu cadastral': '1 311 €', 'Toilette': '1', 'Cave': 'Oui', 'Type de chauffage': 'Gaz'}


In [62]:
detail = driver.find_element_by_tag_name("dt")
print(detail.text)

Prix


In [63]:
"""

1. make a dataset
    1. Initializing it with correct column names.
-
2. open webpage
3. extract the list as dict
-
4. write to dataset
5. keep doing this
6. write dataset to CSV

"""

'\n\n1. make a dataset\n    1. Initializing it with correct column names.\n-\n2. open webpage\n3. extract the list as dict\n-\n4. write to dataset\n5. keep doing this\n6. write dataset to CSV\n\n'

In [64]:
import pandas as pd 



In [70]:
def readAdPage(inp_url):
    """
    Method the reads all the information on the page and writes it to a dict using the key, 
    value pairs used on the website in question. 
    
    :inp_url: get the input URL that we will be gathering information from.
    :return: a dict that stores scraped data
    """
    
    # initialize some variables.
    detailsKeys = []
    detailsValues = []
    details = {}
    
    # grab the URL page code
    driver.get(inp_url)
    
    # find the tables in the page and split data in 2 types, keys and values.
    for desc_list in driver.find_elements_by_tag_name("dt"):
        detail = desc_list.text.split("\n")
        detailsKeys.extend(detail)

    for desc_list in driver.find_elements_by_tag_name("dd"):
        detail = desc_list.text.split("\n")
        detailsValues.extend(detail)

    # build a dict containing all gathered data
    for idx, x in enumerate(detailsKeys):
        details[x] = detailsValues[idx]
        
    return details

In [68]:
# we start with a list-of-dicts and append the found lists to it

data: List[Dict] = []

In [71]:
result = readAdPage("https://immoelissa.be/annonces-immobilieres/4295347-rez-de-chaussee-commercial-3/")



In [72]:
data.append(result)
data.append(result)
data.append(result)


In [73]:
print(data)

[{'Prix': '105 000 €', 'Référence': '4045413', 'Ville': 'Liege', 'Type du bien': 'Commerce', 'Disponibilité': 'Vendu', 'État du bien': 'Bon état', 'Revenu cadastral': '1 311 €', 'Toilette': '1', 'Cave': 'Oui', 'Type de chauffage': 'Gaz'}, {'Prix': '105 000 €', 'Référence': '4045413', 'Ville': 'Liege', 'Type du bien': 'Commerce', 'Disponibilité': 'Vendu', 'État du bien': 'Bon état', 'Revenu cadastral': '1 311 €', 'Toilette': '1', 'Cave': 'Oui', 'Type de chauffage': 'Gaz'}, {'Prix': '105 000 €', 'Référence': '4045413', 'Ville': 'Liege', 'Type du bien': 'Commerce', 'Disponibilité': 'Vendu', 'État du bien': 'Bon état', 'Revenu cadastral': '1 311 €', 'Toilette': '1', 'Cave': 'Oui', 'Type de chauffage': 'Gaz'}]


In [76]:
driver.get('https://immoelissa.be/immobilier/?sort=prix-c')

In [118]:
def properties_url(inp_url):
    driver.get(inp_url)
    url = []
    time.sleep(random.uniform(1.0, 2.0))
    for elem in driver.find_elements_by_tag_name("a"):
        link = elem.get_attribute("href")
        if not link is None and "annonces" in link:
            url.append(link)
    return url

urls = properties_url('https://immoelissa.be/immobilier/?sort=prix-c')
print(urls)

['https://immoelissa.be/annonces-immobilieres/4295347-rez-de-chaussee-commercial-3/', 'https://immoelissa.be/annonces-immobilieres/4336908-agreable-appartement/', 'https://immoelissa.be/annonces-immobilieres/4345864-agreable-appartement/', 'https://immoelissa.be/annonces-immobilieres/4393872-studio-bien-situe/', 'https://immoelissa.be/annonces-immobilieres/4342411-maison-a-liege/', 'https://immoelissa.be/annonces-immobilieres/4349281-agreable-maison/', 'https://immoelissa.be/annonces-immobilieres/4398654-agreable-appartement/', 'https://immoelissa.be/annonces-immobilieres/4099146-vendu/', 'https://immoelissa.be/annonces-immobilieres/4056078-vendu-en-2-heures-vendu-en-2-heures/']


In [121]:
def readPagination():
    """
    readPagination gathers all pages that contain independent ads. The first one is hardcoded and 
    aferwards it formats the URL as a string until no new ads are found.
    
    :return: a list of URLs that link to each page of a search query. 
    """

    # initiaze a list for storing all the page URLS
    page_urls = []
    first_page = 'https://immoelissa.be/immobilier/?sort=prix-c'
    page_urls.append(first_page)
    
    # initialize a list for storing all the seperate links to individual ads.
    all_advert_urls = []
    advert_urls = []
    counter = 2
       
    advert_urls = properties_url(first_page)
    all_advert_urls.extend(advert_urls)
    
    print(advert_urls)

    while advert_urls != []:
        url2 = f"https://immoelissa.be/immobilier/?pg={counter}&sort=prix-c"
        advert_urls = properties_url(url2)
        all_advert_urls.extend(advert_urls)
        counter += 1
        print(advert_urls)
        
    return all_advert_urls

all_advert_urls = readPagination()

['https://immoelissa.be/annonces-immobilieres/4295347-rez-de-chaussee-commercial-3/', 'https://immoelissa.be/annonces-immobilieres/4336908-agreable-appartement/', 'https://immoelissa.be/annonces-immobilieres/4345864-agreable-appartement/', 'https://immoelissa.be/annonces-immobilieres/4393872-studio-bien-situe/', 'https://immoelissa.be/annonces-immobilieres/4342411-maison-a-liege/', 'https://immoelissa.be/annonces-immobilieres/4349281-agreable-maison/', 'https://immoelissa.be/annonces-immobilieres/4398654-agreable-appartement/', 'https://immoelissa.be/annonces-immobilieres/4099146-vendu/', 'https://immoelissa.be/annonces-immobilieres/4056078-vendu-en-2-heures-vendu-en-2-heures/']
['https://immoelissa.be/annonces-immobilieres/4043951-maison-2-chambres-avec-jardin/', 'https://immoelissa.be/annonces-immobilieres/4455455-agreable-maison/', 'https://immoelissa.be/annonces-immobilieres/4010723-vendu/', 'https://immoelissa.be/annonces-immobilieres/4231794-vendu/', 'https://immoelissa.be/annonc

In [123]:
print(len(all_advert_urls))

91


In [127]:
data = []
x = 0

for url in all_advert_urls:
    result = readAdPage(url)
    data.append(result)
    print(f'working {x}')
    x += 1
    
print(data)

working 0
working 1
working 2
working 3
working 4
working 5
working 6
working 7
working 8
working 9
working 10
working 11
working 12
working 13
working 14
working 15
working 16
working 17
working 18
working 19
working 20
working 21
working 22
working 23
working 24
working 25
working 26
working 27
working 28
working 29
working 30
working 31
working 32
working 33
working 34
working 35
working 36
working 37
working 38
working 39
working 40
working 41
working 42
working 43
working 44
working 45
working 46
working 47
working 48
working 49
working 50
working 51
working 52
working 53
working 54
working 55
working 56
working 57
working 58
working 59
working 60
working 61
working 62
working 63
working 64
working 65
working 66
working 67
working 68
working 69
working 70
working 71
working 72
working 73
working 74
working 75
working 76
working 77
working 78
working 79
working 80
working 81
working 82
working 83
working 84
working 85
working 86
working 87
working 88
working 89
working 90
[{'': '40

In [130]:
database = pd.DataFrame(data)

In [131]:
print(database)

                        Ville Type du bien Disponibilité  État du bien  \
0     4045413           Liege     Commerce         Vendu      Bon état   
1   109 000 €           Liege  Appartement         Vendu  à rafraîchir   
2   109 000 €           Liege  Appartement         Vendu  à rafraîchir   
3     4337874           Liege  Appartement         Vendu      Bon état   
4   109 000 €           Liege       Maison         Vendu      Bon état   
..        ...             ...          ...           ...           ...   
86   89 000 €  Grace-Hollogne      Terrain      À vendre           NaN   
87    2021392           Liege  Appartement         Vendu      Bon état   
88   99 000 €         Herstal  Appartement         Vendu      Bon état   
89   99 000 €           Liege       Maison         Vendu  à rafraîchir   
90   99 000 €           Trooz       Maison         Vendu      Bon état   

   Revenu cadastral Toilette Cave   Type de chauffage Référence  ...  Terrain  \
0           1 311 €        1  

In [133]:
database.to_csv('eliza.csv')

In [134]:
database['source'] = all_advert_urls

In [135]:
database.to_csv('eliza.csv')

In [4]:
import pandas as pd

database = pd.read_csv('eliza.csv')

In [13]:
for index, row in database.iterrows():
    if row['Cave'] == 'Oui':
        ifor_val = 1
    else:
        ifor_val = 0
    database.at[index,'Cave'] = ifor_val

        
for index, row in database.iterrows():
    print(row['Cave'])

1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
1
0
0
1
0
1
0
1
1
0
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
0
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
0
1
0
0
0
1
1
0
1
0
0
0
1
0
0
0
0
0
1
0
1


In [ ]:
class Elissa():
    def __init__(self):
        self.first_page = 'https://immoelissa.be/immobilier/?sort=prix-c'
        self.data = []
    
    def scrape(self):
        """
        Scrape methods calls other methods in the class in order. TODO add threading support.
        """
        
        for url in all_advert_urls:
            result = readAdPage(url)
            self.data.append(result)
    
    def properties_url(self, inp_url):
        driver.get(inp_url)
        url = []
        time.sleep(random.uniform(1.0, 2.0))
        for elem in driver.find_elements_by_tag_name("a"):
            link = elem.get_attribute("href")
            if not link is None and "annonces" in link:
                url.append(link)
        return url
    
    def readPagination(self):
        """
        readPagination gathers all pages that contain independent ads. The first one is hardcoded and 
        aferwards it formats the URL as a string until no new ads are found.

        :return: a list of URLs that link to each page of a search query. 
        """

        # initiaze a list for storing all the page URLS
        page_urls = []
        page_urls.append(self.first_page)

        # initialize a list for storing all the seperate links to individual ads.
        all_advert_urls = []
        advert_urls = []
        counter = 2

        advert_urls = properties_url(self.first_page)
        all_advert_urls.extend(advert_urls)

        while advert_urls != []:
            url2 = f"https://immoelissa.be/immobilier/?pg={counter}&sort=prix-c"
            advert_urls = properties_url(url2)
            all_advert_urls.extend(advert_urls)
            counter += 1
            print(advert_urls)
        
        return all_advert_urls
    
    def readAdPage(self, inp_url):
        """
        Method the reads all the information on the page and writes it to a dict using the key, 
        value pairs used on the website in question. 

        :inp_url: get the input URL that we will be gathering information from.
        :return: a dict that stores scraped data
        """

        # initialize some variables.
        detailsKeys = []
        detailsValues = []
        details = {}

        # grab the URL page code
        driver.get(inp_url)

        # find the tables in the page and split data in 2 types, keys and values.
        for desc_list in driver.find_elements_by_tag_name("dt"):
            detail = desc_list.text.split("\n")
            detailsKeys.extend(detail)

        for desc_list in driver.find_elements_by_tag_name("dd"):
            detail = desc_list.text.split("\n")
            detailsValues.extend(detail)

        # build a dict containing all gathered data
        for idx, x in enumerate(detailsKeys):
            details[x] = detailsValues[idx]

        return details
